<a href="https://colab.research.google.com/github/chaodi51/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [20]:
# YOUR CODE HERE
# %%writefile get_data.py
import os
import zipfile
from pathlib import Path
import requests
import io

# setup download folder
data_path = Path('./data')
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if not image_path.is_dir():
    print(f"{image_path} directory does not exist, making one.")
    image_path.mkdir(exist_ok=True, parents=True)
else:
    print(f"{image_path} directory exists.")

# Download pizza, steak, sushi data
response = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip", stream=True)
# Unzip pizza, steak, sushi data
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(image_path)
    print(f"Files extracted to {image_path}")


data/pizza_steak_sushi directory exists.
Files extracted to data/pizza_steak_sushi


In [2]:
# Example running of get_data.py
!python get_data.py

data/pizza_steak_sushi directory does not exist, making one.
Files extracted to data/pizza_steak_sushi


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [6]:
# make the modules folder, which will host the follows:
"""
data_setup.py - a file to prepare and download data if needed.
engine.py - a file containing various training functions.
model_builder.py - a file to create a PyTorch TinyVGG model.
train.py - a file to leverage all other files and train a target PyTorch model.
utils.py - a file dedicated to helpful utility functions.
Extra: predictions.py - a file for making predictions with a trained PyTorch model and input image model_builder.py, utils.py and engine.py
"""

import os
from pathlib import Path

module_dir = Path('./going_modular')
module_dir.mkdir(parents=True, exist_ok=True)
file_path = module_dir / 'data_step.py'

In [16]:
str(module_dir / 'data_step.py')

'going_modular/data_step.py'

In [47]:

%%writefile going_modular/data_step.py

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
from typing import Tuple, List
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
) -> Tuple[DataLoader, DataLoader, List[str]]:

    """Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
      train_dir: Path to training directory.
      test_dir: Path to testing directory.
      transform: torchvision transforms to perform on training and testing data.
      batch_size: Number of samples per batch in each of the DataLoaders.
      num_workers: An integer for number of workers per DataLoader.

    Returns:
      A tuple of (train_dataloader, test_dataloader, class_names).
      Where class_names is a list of the target classes.
      Example usage:
        train_dataloader, test_dataloader, class_names = \
          = create_dataloaders(train_dir=path/to/train_dir,
                              test_dir=path/to/test_dir,
                              transform=some_transform,
                              batch_size=32,
                              num_workers=4)
    """

    # load training and testing datasets
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # get class names
    class_names = train_data.classes

    # creat DataLoader
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_step.py


In [48]:
!python going_modular/data_step.py

In [35]:
!touch going_modular/__init__.py

In [53]:
# build the train.py script
%%writefile going_modular/train.py
"""
Train an image classifier using pre-compiled modules
"""
train_path = 'data/pizza_steak_sushi/train'
test_path = 'data/pizza_steak_sushi/test'

import torch
from torchvision import transforms
import argparse
import os
from data_step import create_dataloaders
# engine, model_builder, utils


# Setup default hyperparameters
parser = argparse.ArgumentParser(description="Train a model with customizable hyperparameters.")

# Add arguments with default values
parser.add_argument('--num_epochs', type=int, default=5, help='Number of training epochs (default: 5)')
parser.add_argument('--batch_size', type=int, default=32, help='Batch size for training (default: 32)')
parser.add_argument('--hidden_units', type=int, default=10, help='Number of hidden units in the model (default: 10)')
parser.add_argument('--learning_rate', type=float, default=0.001, help='Learning rate for training (default: 0.001)')

# Parse the command-line arguments
args = parser.parse_args()

NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# prepare batched datasets using dataloader in data_step.py
train_data, test_data, class_names = data_step.create_dataloaders(
  train_dir=train_path,
  test_dir=test_path,
  transform=data_transform,
  batch_size=BATCH_SIZE)

# build model using model_builder.py


# train the model with help of engine.py

#

Overwriting going_modular/train.py


In [54]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

['pizza', 'steak', 'sushi']


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [ ]:
# YOUR CODE HERE

In [ ]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg